In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image
from torchvision import transforms
import torch
import torch.optim as optim
import torch.nn as nn



In [2]:
DATA_DIR = "data"        # pasta com as duas classes
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
DEVICE = "cuda" 

In [ ]:
# Normalização padrão ImageNet
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

# Transform padrão para ResNet50 (sem data augmentation)
IMAGENET_transform = transforms.Compose([
    transforms.Resize(256),           # aumenta a menor dimensão para 256
    transforms.CenterCrop(224),       # corta para 224x224
    transforms.ToTensor(),            # converte PIL->Tensor e escala [0,1]
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])

In [37]:
#Definindo a classe do dataset
class ImageDataset(Dataset):
    
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform or default_transform  # se não passar transform, usa padrão
        self.samples = []  # [(img_path, label_idx), ...]
        self.class_to_idx = {}  # {"classe": idx}

        # Cria lista de imagens + label
        for i, class_dir in enumerate(sorted(p for p in self.root_dir.iterdir() if p.is_dir())):
            self.class_to_idx[class_dir.name] = i
            for img_path in class_dir.glob('*'):
                self.samples.append((img_path, i))

    def __len__(self):
        return len(self.samples)

    def __str__(self):
        return f"ImageDataset with {len(self)} samples from {len(self.class_to_idx)} classes."


    def __getitem__(self, idx):
        img_path, label = self.samples[idx]

        # Abre a imagem de forma segura
        with open(img_path, 'rb') as f:
            img = Image.open(f).convert('RGB')  # garante 3 canais

        # Aplica transformações (pré-processamento/resnet normalization)
        img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.long)

In [38]:
# Dividindo dataset em treino e validação
from torch.utils.data import DataLoader, random_split

dataset = ImageDataset("data\DataSet")  # pasta com as 2 classes

train_size = int(0.8 * len(dataset))
val_size   = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [39]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)


In [ ]:
# Modelo ResNet50
num_classes = len(dataset.class_to_idx)
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)  # pretreinada
model.fc = nn.Linear(model.fc.in_features, num_classes)                  # ajusta a última camada para 2 classes
model = model.to("cuda") #Escolha de onde rodar o modelo

#Escolha do normalizador, otimizador e LR
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [42]:
#treino e validação

for epoch in range(1, EPOCHS + 1):

    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(DEVICE, non_blocking=True)
        labels = labels.to(DEVICE, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss /= total
    train_acc = correct / total

    # ---- Validação ----
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(DEVICE, non_blocking=True)
            labels = labels.to(DEVICE, non_blocking=True)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss /= total
    val_acc = correct / total

    print(f"Epoch {epoch}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

# ------------------------------
# 7️⃣ Salvar modelo
# ------------------------------
torch.save(model.state_dict(), "resnet50_finetuned.pth")
print("Modelo salvo em resnet50_finetuned.pth")

Epoch 1/10 | Train Loss: 0.1388 Acc: 0.9418 | Val Loss: 0.0566 Acc: 0.9862
Epoch 2/10 | Train Loss: 0.0298 Acc: 0.9891 | Val Loss: 0.0593 Acc: 0.9803
Epoch 3/10 | Train Loss: 0.0160 Acc: 0.9965 | Val Loss: 0.0422 Acc: 0.9822
Epoch 4/10 | Train Loss: 0.0038 Acc: 1.0000 | Val Loss: 0.0454 Acc: 0.9822
Epoch 5/10 | Train Loss: 0.0035 Acc: 1.0000 | Val Loss: 0.0447 Acc: 0.9822
Epoch 6/10 | Train Loss: 0.0064 Acc: 0.9985 | Val Loss: 0.0563 Acc: 0.9842
Epoch 7/10 | Train Loss: 0.0325 Acc: 0.9896 | Val Loss: 0.0604 Acc: 0.9862
Epoch 8/10 | Train Loss: 0.0254 Acc: 0.9921 | Val Loss: 0.0437 Acc: 0.9822
Epoch 9/10 | Train Loss: 0.0127 Acc: 0.9965 | Val Loss: 0.0695 Acc: 0.9822
Epoch 10/10 | Train Loss: 0.0032 Acc: 1.0000 | Val Loss: 0.0504 Acc: 0.9862
Modelo salvo em resnet50_finetuned.pth


In [ ]:
from Config import train_and_validate

# Definir modelo, dataloaders, criterion, optimizer, etc.
MODEL_NAME = "resnet50" # Nome do modelo
SAVE_PATH = f"{MODEL_NAME}_finetuned.pth"

train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=DEVICE,
    epochs=EPOCHS,
    save_path=SAVE_PATH
)